<a href="https://colab.research.google.com/github/KczBen/tol403-lokaverkefni/blob/main/Lokaverkefni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone repo data into environment
!git clone https://github.com/KczBen/tol403-lokaverkefni.git

Cloning into 'tol403-lokaverkefni'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 2), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 490.79 KiB | 1.70 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [12]:
class Node:
  def __init__(self, osmid, x, y, primary):
    self.osmid = osmid,
    self.x = x,
    self.y = y,
    self.primary = primary

  def __repr__(self):
        return f"Node(id={self.osmid}, x={self.x}, y={self.y}, primary={self.primary})"

class Edge:
  def __init__(self, u, v, length, name):
    self.u = u,
    self.v = v,
    self.length = length,
    self.name = name

  def __repr__(self):
        return f"Edge(u={self.u}, v={self.v}, length={self.length}, name={self.name})"

In [13]:
import csv

def load_csv_into_list(filepath, cls):
    with open(filepath, 'r') as file:
        reader = csv.DictReader(file, delimiter = "\t")
        objects = []
        for row in reader:
            obj = cls(
                **{k: v for k, v in row.items()}
            )
            objects.append(obj)
        return objects


nodes = load_csv_into_list('tol403-lokaverkefni/data/nodes.tsv', Node)
edges = load_csv_into_list('tol403-lokaverkefni/data/edges.tsv', Edge)

Node(id=('12885866',), x=('-21.9374735',), y=('64.0672714',), primary=False)
Node(id=('12885876',), x=('-21.9399509',), y=('64.0761569',), primary=True)
Node(id=('12885922',), x=('-22.0878904',), y=('64.0319775',), primary=False)
Node(id=('12885923',), x=('-21.9378515',), y=('64.0672598',), primary=False)
Node(id=('12885924',), x=('-21.9380784',), y=('64.0670465',), primary=True)
Node(id=('12885930',), x=('-21.9406337',), y=('64.0759348',), primary=True)
Node(id=('12885937',), x=('-22.036558',), y=('64.0414971',), primary=False)
Node(id=('12885952',), x=('-21.9401375',), y=('64.0762733',), primary=True)
Node(id=('12885974',), x=('-21.9379271',), y=('64.0669345',), primary=False)
Node(id=('12885979',), x=('-21.9406832',), y=('64.0762644',), primary=True)
